In [1]:
from solver_VesselNET import Solver
import torch.nn.functional as F
import torchvision.models as models
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
from torchvision import transforms, datasets
#from classifiers.KaggleNet import KaggleNET
from classifiers.VesselNET import VesselNET

from data_utils_VesselNET import SegmentationData

%load_ext autoreload
%autoreload 2

In [2]:
data_transform = transforms.Compose([
        transforms.Resize([128, 128]),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip()
    ])

train_data = SegmentationData(image_paths_file='../data/image_files.txt', transform = data_transform, mode = 'train')
train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=12,
                                           shuffle=True,
                                           num_workers=4)

val_data = SegmentationData(image_paths_file='../data/image_files.txt',  transform = data_transform, mode = 'val')
val_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=12,
                                           shuffle=True,
                                           num_workers=4)

In [3]:
model = VesselNET(num_classes = 2)
#model = torch.load("models/KaggleNet_v3.model")

In [4]:
#for param in list(model.parameters()):
#    param.requires_grad = False
    
#for param in list(model.parameters())[-2::]:
#    param.requires_grad = True


solver = Solver(optim_args={"lr": 0.001, #0.0025, #1.e-3, #1.e-2
                            "betas": (0.9, 0.999),
                            "eps": 1e-8,
                            "weight_decay": 0.},
                loss_func = torch.nn.BCELoss())

#print(model)
solver.train(model, train_loader, val_loader, log_nth=10, num_epochs=5)

START TRAIN.


/home/yy/anaconda3/lib/python3.6/site-packages/torch/nn/modules/container.py:67: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


[Iteration 10/1410] TRAIN loss: 0.691


Process Process-3:
Process Process-2:
Process Process-4:
Process Process-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/yy/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/yy/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/yy/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yy/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/yy/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/yy/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yy/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in ru

KeyboardInterrupt: 

In [5]:
model.save("models/VesselNET1.model")

Saving model... models/VesselNET1.model


NameError: name 'torch' is not defined

In [ ]:
test_data_transform = transforms.Compose([
        transforms.Resize([128, 128])
    ])


test_data = SegmentationData(image_paths_file='../data/image_files.txt',  transform = data_transform, mode = 'test')
test_loader = torch.utils.data.DataLoader(test_data,
                                          batch_size=1,
                                          shuffle=False,
                                          num_workers=1)

In [ ]:
img, target = test_data[25]

inputs = img.unsqueeze(0)
inputs = Variable(inputs)

outputs = model.forward(inputs)
_, preds = torch.max(outputs, 1)
pred = preds[0].data.cpu()

img=np.squeeze(img)
target = target.numpy()
pred = pred.numpy()

fig, ax = plt.subplots(1, 3)
ax[0].imshow(img)
ax[0].axis('off')
ax[1].imshow(target)
ax[1].axis('off')
ax[2].imshow(pred)
ax[2].axis('off')
plt.show()

In [ ]:
num_example_imgs = 4
indices = np.random.randint(test_data.__len__(), size = num_example_imgs)
plt.figure(figsize=(15, 5 * num_example_imgs))
for i, (img, target) in enumerate(test_data[20:24]):

    inputs = img.unsqueeze(0)
    inputs = Variable(inputs)
    
    outputs = model.forward(inputs)
    _, preds = torch.max(outputs, 1)
    pred = preds[0].data.cpu()
    
    img=np.squeeze(img)
    target = target.numpy()
    pred = pred.numpy()
    
    # img
    plt.subplot(num_example_imgs, 3, i * 3 + 1)
    plt.axis('off')
    plt.imshow(img)
    if i == 0:
        plt.title("Input image")
    
    # target
    plt.subplot(num_example_imgs, 3, i * 3 + 2)
    plt.axis('off')
    plt.imshow(target)
    if i == 0:
        plt.title("Target image")

    # pred
    plt.subplot(num_example_imgs, 3, i * 3 + 3)
    plt.axis('off')
    plt.imshow(pred)
    if i == 0:
        plt.title("Prediction image")
    
plt.show()

In [ ]:
test_scores = []
model.eval()
for inputs, targets in test_loader:
    inputs, targets = Variable(inputs), Variable(targets)
    if model.is_cuda:
        inputs, targets = inputs.cuda(), targets.cuda()
    
    outputs = model.forward(inputs)
    _, preds = torch.max(outputs, 1)
    targets_mask = targets >= 0
    test_scores.append(dice_coefficient(np.squeeze(targets.data.numpy()), np.squeeze(preds.data.numpy())))#np.mean((preds == targets)[targets_mask].data.cpu().numpy()))
    
model.train()
np.mean(test_scores)

In [ ]:
np.unique(test_scores)

In [ ]:
def dice_coefficient(ground_truth, predicted):
    gt = ground_truth
    p = predicted
    if np.sum(p) + np.sum(gt) == 0:
        return 1
    else:
        dice = np.sum(p[gt==1])*2.0 / (np.sum(p) + np.sum(gt))
        return dice